In [ ]:
import os

notebook_dir = os.getcwd()
# Navigate to ipmof library directory
os.chdir(os.path.join(os.getcwd(), '..', '..'))
from ipmof.crystal import MOF
from ipmof.forcefield import read_ff_parameters
from ipmof.energymap import energy_map, energy_map_atom_list
from ipmof.parameters import read_parameters
from ipmof.interpenetration import run_interpenetration, get_interpenetration_list

## Reading .cif files into MOF object using [ASE](https://wiki.fysik.dtu.dk/ase/) library

In [ ]:
# Select MOF directory
supercell_dir = os.path.join(notebook_dir, 'Supercell')
mof1_path = os.path.join(supercell_dir, 'LEHXUT.cif')
mof2_path = os.path.join(supercell_dir, 'XAMDUM02.cif')

mof1 = MOF(mof1_path)
mof2 = MOF(mof2_path)
print('Reading MOFs: %s + %s' % (mof1.name, mof2.name))

## Initializing simulation parameters and directories

In [ ]:
# Read simulation parameters and directories
sim_par, sim_dir = read_parameters()

# Don't try homo-interpenetration
sim_par['self_interpenetration'] = False

# Read files and export to supercell directory in Notebooks
sim_dir['export_dir'] = supercell_dir
sim_dir['energy_map_dir'] = supercell_dir
sim_dir['mof_dir'] = supercell_dir

# Read excel file containing force field information
force_field = read_ff_parameters(sim_dir['force_field_path'], sim_par['force_field'])

# Calculate atom list according to 'energy_map_atom_list' simulation parameter
atom_list = energy_map_atom_list(sim_par, force_field, mof_path_list)

## Generating energymap

In [ ]:
# Export initialization file containing MOF names and simulation parameters
print('Starting energy map calculation for', mof1.name, 'with grid size:',
      sim_par['grid_size'], '| cut-off radius:', str(sim_par['cut_off']))
print('Atom list ->', atom_list['atom'])
print('Atom list type: %s' % sim_par['energy_map_atom_list'])
print('Energy map(s) will be exported in', sim_par['energy_map_type'], 'format')
print('-' * 80)
print('Calculating energy map for ->', os.path.basename(mof1_path))
print('This will take a while...')
emap = energy_map(sim_par, mof1_path, atom_list, force_field)
print('Done!')

## Running interpenetration test

In [ ]:
emap_path = os.path.join(supercell_dir, '%s_emap.npy' % mof1.name)
emap_mof_path = mof1_path
ip_mof_path = mof2_path
interpenetration_path = (emap_path, emap_mof_path, ip_mof_path)
print('Initializing interpenetration for 1 MOF combination...')
print('-' * 80, '\nEnergy map ->', os.path.basename(emap_path))
print('Interpenetration ->', os.path.basename(ip_mof_path) + '\n' + '-' * 80)

run_interpenetration(interpenetration_path, sim_par, sim_dir)
print('Exporting results to: %s' % sim_dir['export_dir'])
print('Done!')